In [19]:
import ipywidgets as widgets
from ipywidgets import Checkbox, VBox
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

df = vaex.from_csv("testdata.csv")


namesFromFile = ["Tag 1", "Data Structure 2", "Test 3"]
tagsFromFile = [0, 1, 2]
numTags = len(namesFromFile)

selectionList = [*tagsFromFile]


def updateGraph(change):
    if(change.name == 'value'):
        
        name = change.owner.description
        tag = tagsFromFile[namesFromFile.index(name)]

        if(change.new == True):
            selectionList.append(tag)
        else:
            selectionList.remove(tag)
        df.select(True)
        

items = [Checkbox(description=namesFromFile[i], value=True, disabled=False, indent=False) for i in range(0, numTags)]

for i in range(numTags):
    items[i].observe(updateGraph)
    
checks = VBox(items)

selection = df.select(True)
@vaex.jupyter.interactive_selection(df)
def plot(*args, **kwargs):
    df.plot(df.x, df.y, what=np.log(vaex.stat.count()+1), selection=[(df.tag == x) for x in selectionList])
    plt.show()
    
display(checks)

Output()

False
